### **Importing libraries**

In [1]:
import time
import pandas as pd
import numpy as np
from collections import OrderedDict
import math

### **Mouthing to Drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### **Uploading a smaller data set to test the code**

In [3]:
filename = pd.read_csv('/content/drive/My Drive/IIIT_A/DA/sample.csv',header =None,delimiter = ',')
data = np.array(filename)

### **Finding the minimum support value.**

In [4]:
row,col=filename.shape
min_support = math.ceil(0.3*row)
min_support

2

### **Function to create Trees**

In [5]:
global Tree
Tree =1
def createTree(data, minSup): 

    global Tree
    Table = {}
  
    for transaction in data:
        for item in transaction:
            Table[item] = Table.get(item, 0) + data[transaction]

    for k in list(Table):  
        if Table[k] < minSup: 
            del(Table[k])

    freqItemSet = set(Table.keys())
   
    if len(freqItemSet) == 0: return None, None  
    for k in Table:
        Table[k] = [Table[k], None] 
   
    recreatTree = node('Null Set', 1, None) 

    for tranSet, count in data.items(): 
        localD = {}
        for item in tranSet:  
            if item in freqItemSet:
                localD[item] =Table[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, recreatTree, Table, count)
        
        print("\033[1m""Tree pass:""\033[0m",Tree) 
        Tree = Tree+1
        recreatTree.disp()
        print('\n\n')
    return recreatTree, Table 

### **Class node to create tree nodes**

In [6]:
class node:
    def __init__(self, nameValue, itemOccurance, parentNode):
        self.name = nameValue
        self.count = itemOccurance
        self.next = None
        self.parent = parentNode      
        self.children = {} 

            
    def disp(self, space=1):
        print ('..'*space, self.name, ' ', self.count)
        for child_node in self.children.values():
            child_node.disp(space+1)

      
    def increment(self, itemOccurance):
        self.count += itemOccurance


In [7]:
def updatetable(node, nextNode):   
    while (node.next != None):    
        node = node.next
    node.next = nextNode

In [8]:
def updateTree(items, inTree, Table, count):

    if items[0] in inTree.children:
        inTree.children[items[0]].increment(count) 
    else:   
        inTree.children[items[0]] = node(items[0], count, inTree)
        if Table[items[0]][1] == None: 
            Table[items[0]][1] = inTree.children[items[0]]
        else:
            updatetable(Table[items[0]][1], inTree.children[items[0]])


    if len(items) > 1:
        updateTree(items[1::], inTree.children[items[0]], Table, count)

### **To create Initial sets**

In [9]:
def createInitSet(dataSet):
    Dict = OrderedDict()
    for trans in dataSet:
        p = []
        for t in trans:
          if(type(t) == float):
            continue
          p.append(t)
        if frozenset(p) in Dict.keys():
          Dict[frozenset(p)] += 1
        else:
          Dict[frozenset(p)] = 1
    return Dict

In [10]:
initialSetofitems = createInitSet(data)

In [11]:
initialSetofitems

OrderedDict([(frozenset({'buns', 'hotdogs', 'ketchup'}), 1),
             (frozenset({'buns', 'hotdogs'}), 1),
             (frozenset({'chips', 'coke', 'hotdogs'}), 2),
             (frozenset({'chips', 'coke'}), 1),
             (frozenset({'chips', 'ketchup'}), 1)])

### **Tree for each steps displayed **

In [12]:
tree, counttable = createTree(initialSetofitems,min_support)

Tree pass: 1
.. Null Set   1
.... hotdogs   1
...... ketchup   1
........ buns   1



Tree pass: 2
.. Null Set   1
.... hotdogs   2
...... ketchup   1
........ buns   1
...... buns   1



Tree pass: 3
.. Null Set   1
.... hotdogs   4
...... ketchup   1
........ buns   1
...... buns   1
...... chips   2
........ coke   2



Tree pass: 4
.. Null Set   1
.... hotdogs   4
...... ketchup   1
........ buns   1
...... buns   1
...... chips   2
........ coke   2
.... chips   1
...... coke   1



Tree pass: 5
.. Null Set   1
.... hotdogs   4
...... ketchup   1
........ buns   1
...... buns   1
...... chips   2
........ coke   2
.... chips   2
...... coke   1
...... ketchup   1





### **final FP tree**

In [13]:
tree.disp()

.. Null Set   1
.... hotdogs   4
...... ketchup   1
........ buns   1
...... buns   1
...... chips   2
........ coke   2
.... chips   2
...... coke   1
...... ketchup   1


### **Count of each item**

In [14]:
counttable

{'buns': [2, <__main__.node at 0x7fd3bef30650>],
 'chips': [4, <__main__.node at 0x7fd3ba600d50>],
 'coke': [3, <__main__.node at 0x7fd3ba600c90>],
 'hotdogs': [4, <__main__.node at 0x7fd3bef30c50>],
 'ketchup': [2, <__main__.node at 0x7fd3bef30950>]}